In [1]:
import socket
import cv2
import numpy as np
import imutils
import time

In [2]:
#加载LSTM神经网络加速器和DMA接口
from pynq import Overlay, allocate
import pynq.lib.dma
from pynq import allocate


overlay = Overlay('./LSTM2.bit')  # 加载Overlay
dma = overlay.axi_dma_0  # 定义DMA对象

In [5]:
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)


videoIn = cv2.VideoCapture()
videoIn.open(0)
videoIn.set(3,160)
videoIn.set(4,120)
#import face data
face_cascade = cv2.CascadeClassifier(
        '/home/xilinx/jupyter_notebooks/base/video/data/'
        'haarcascade_frontalface_default.xml')
        #import eye data
mouth_cascade = cv2.CascadeClassifier(
        '/home/xilinx/jupyter_notebooks/base/video/data/'
        'haarcascade_mcs_mouth.xml')

test_images = allocate(shape=(784,), dtype=np.float32)

IMG_NUM = 1
index = [0] * IMG_NUM

while True:
    ret, frame_vga = videoIn.read()
    if (ret):   
# 发送数据:
        # picture thransform to gray
        gray = cv2.cvtColor(frame_vga, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        # detect face and eyes
        
        
        time_start = time.time()  # 记录开始时间
                # function()   执行的程序

        for (x,y,w,h) in faces:
            #draw rectangle
            mh = int(0.6 * h)
            cv2.rectangle(frame_vga,(x,y + mh),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y + mh:y + h, x:x+w]
            roi_color = frame_vga[y + mh:y + h, x:x+w] 
            test = gray[y + mh:y + h, x :x+w]
            mouths = mouth_cascade.detectMultiScale(roi_gray)
        
            for (ex,ey,ew,eh) in mouths:
                cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                test = test[ey:ey + eh, ex :ex+ew]
                
                
                time_start = time.time()  # 记录开始时间
                test1=cv2.resize(test,(28,28))
                test1 = test1.flatten()
#                print(test1)
                for i in range(784):
                    test_images[i] = test1[i]
#                print(test_images)
                #print(test_images.physical_address)
                out_buf = allocate(shape=(10000, 10), dtype = np.float32)
                for i in range (IMG_NUM):
                    dma.sendchannel.transfer(test_images)  # 调用DMA将待预测图片数据传输到IP核
                    dma.recvchannel.transfer(out_buf[index[i]])  # 调用DMA从IP核获取RNN的推导结果
                    dma.sendchannel.wait()  # 等待DMA发送完成
                    dma.recvchannel.wait()  # 等待DMA接收完成

                out_list = np.array(out_buf[index[i]]).tolist()
                max_indx = out_list.index(max(out_list))  # 推导结果向量的最大分量的下标即为预测结果
                print('Result: %d,' % max_indx)
                time_end = time.time()  # 记录结束时间
                time_sum = time_end - time_start  # 计算的时间差为程序的执行时间，单位为秒/s
                print(time_sum)
                
#                 out_buf = allocate(shape=(1, 5), dtype = np.float32)
#                 dma.sendchannel.transfer(test_images)  # 调用DMA将待预测图片数据传输到IP核
#                 dma.recvchannel.transfer(out_buf)  # 调用DMA从IP核获取RNN的推导结果
#                 dma.sendchannel.wait()  # 等待DMA发送完成
#                 dma.recvchannel.wait()  # 等待DMA接收完成



                    
#                 photoname+=1
#                 filename = str(photoname) + '.jpg'  # filename为图像名字，将photoname作为编号命名保存的截图
#                 
#                 cv2.imwrite(filename, test1)  # 截图 前面为放在桌面的路径 frame为此时的图像              
#                 print(filename + '保存成功')  # 打印保存成功
#                 if(photoname>20):
#                     print(photoname)
#                     break


    
        frame_vga=imutils.resize(frame_vga, width=480)
        img_encode = cv2.imencode('.jpg', frame_vga)[1]
        data_encode = np.array(img_encode)
        data = data_encode.tostring()
        s.sendto(data, ('192.168.137.1', 6050))
        



#                 photoname+=1
#                 filename = str(photoname) + '.jpg'  # filename为图像名字，将photoname作为编号命名保存的截图
#                 cv2.imwrite(filename, frame_vga)  # 截图 前面为放在桌面的路径 frame为此时的图像
#                 print(filename + '保存成功')  # 打印保存成功


# #             if(i>5):  
# #                 #print(eyes.shape)
# #                 #------------------------------------------------
# #                 #test resize
# #                 frame_vga=cv2.resize(test1,(1024,768))
# #     #           print(img.shape)
# #                 b,g,r = cv2.split(test1)
# #                 img_rgb = cv2.merge([r,g,b])
# #                 cv2.imwrite('31.jpg',test1)
# #                 print("image save")
# #                 #break while when detected face
# #                 #------------------------------------------------   
# #                 face_flag = False
# #                 break


#         # print the result picture
#             get_ipython().magic('matplotlib inline') 
#             plt.imshow(np_frame[:,:,[2,1,0]])
#             plt.show()

        
# 接收数据:

        #print(s.recv(1024).decode('utf-8'))

s.close()

<ipython-input-5-2d1d8a6726b9>:94: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  data = data_encode.tostring()


Result: 1,
0.04680991172790527
Result: 1,
0.04445385932922363
Result: 1,
0.04734611511230469
Result: 1,
0.046861886978149414
Result: 1,
0.047846317291259766
Result: 1,
0.042680978775024414
Result: 1,
0.047611236572265625
Result: 1,
0.04475855827331543
Result: 1,
0.045369863510131836
Result: 1,
0.043358802795410156
Result: 1,
0.0448453426361084
Result: 1,
0.044481754302978516
Result: 1,
0.045091867446899414
Result: 1,
0.04392552375793457
Result: 1,
0.04294419288635254
Result: 1,
0.04464221000671387
Result: 1,
0.043347835540771484
Result: 1,
0.04296731948852539
Result: 1,
0.044808149337768555
Result: 1,
0.05166888236999512
Result: 1,
0.04492950439453125
Result: 1,
0.043512582778930664
Result: 1,
0.04292750358581543
Result: 1,
0.04564809799194336
Result: 1,
0.04316973686218262
Result: 1,
0.04368090629577637
Result: 1,
0.04482913017272949
Result: 1,
0.04309368133544922
Result: 1,
0.06358885765075684
Result: 1,
0.045794010162353516
Result: 1,
0.0458836555480957
Result: 1,
0.0431115627288818

KeyboardInterrupt: 